# Getting Started with Pyomo

[Pyomo](http://www.pyomo.org/) is a state-of-the-art language for solving optimization problems embedded within Python. Using Pyomo, a user can describe optimization model by specifying decision **variables**, **constraints**, and an optimization **objective**. Pyomo includes features to enable modeling, specifying a solver, and displaying the solution.

## Installation

Provided you have previously installed the Anaconda package, the following commands should successfully install Pyomo, and the glpk, ipopt, and COIN-OR cbc solvers.  These commands should be executed in a command window on either a Mac or PC.

    conda install -c conda-forge pyomo
    conda install -c conda-forge pyomo.extras
    conda install -c conda-forge glpk
    conda install -c conda-forge ipopt
    conda install -c conda-forge coincbc

This installation provides a capable open-source optimization suite with multiple solvers. The solvers are

* [glpk](https://en.wikibooks.org/wiki/GLPK). "GNU Linear Programming Kit" is a software package written in highly portable C for the solution of mixed integer linear programming and related problems.
* [ipopt](https://en.wikipedia.org/wiki/IPOPT). "Interior Point Optimizer" for large scale nonlinear optimization of continuous systems. 
* [cbc](https://projects.coin-or.org/Cbc). "COIN-OR Branch and Cut" is a mixed integer linear programming solver written in C++. It generally solves the same problems as glpk, but can run multiple threads, and is usually faster and more robust.

This collection of solvers is well suited a wide range of process applications. This suite can be further augmented by installing additional solvers from open-source and commercial sources.

## Example: Solving a System of Equations

### Using Scalar Parameters and Variables

A fully specified system of linear equations given by

\begin{align}
3x + 4y & = 26 \\
2x - 3y & = - 11
\end{align}

can be readily solved as a convex optimization problem. The following cell demonstrates the specification and solution of these equations using $x$ and $y$ as decision variables.

The key idea behind Pyomo is the creation of a **model** that includes **variables**, **constraints**, and an **objective**. The objective specifies the a performance measure that is to be minimized (default) or maximized. A **solver** is then created to solve the model.

In [1]:
from pyomo.environ import *

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.x = Var()
model.y = Var()

# add model constraints
model.constraints = ConstraintList()
model.constraints.add(3*model.x + 4*model.y ==  26)
model.constraints.add(2*model.x - 3*model.y == -11)

# add a model objective
model.objective = Objective(expr = model.x + model.y)

# create a solver
solver = SolverFactory('glpk')

# solve
solver.solve(model)

# print solutions
print("x =", model.x.value)
print("y =", model.y.value)

x = 2.0
y = 5.0


### Pretty print model solution

In [2]:
model.pprint()

1 Set Declarations
    constraints_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        [1, 2]

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   2.0 :  None : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   5.0 :  None : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x + y

1 Constraint Declarations
    constraints : Size=2, Index=constraints_index, Active=True
        Key : Lower : Body      : Upper : Active
          1 :  26.0 : 3*x + 4*y :  26.0 :   True
          2 : -11.0 : 2*x - 3*y : -11.0 :   True

5 Declarations: x y constraints_index constraints objective


## Python Lists, Sets, Dictionaries, and Iterators

Pyomo is integrated with the Python language, and inherits significant functionality by leveraging the basic data structures of Python. To make the best use of Pyomo, it is important to understand the basics of Python lists, sets, and dictionaries.

### Data in Matrix/Vector Format

The example above used scalar modeling components, `model.x = Var()` and `model.y = Var()`, and each constraint was added as a separate line in the model.  This is fine for small problems with a just a few unknowns, but becomes impractical for larger problems.

Here we repeat the example above, this time using `numpy` arrays to enter the data. An indexed variable `model.x` represents the unknowns, and the constraints are indexed as well. The indices are represented by the Python `range()` statement.

In [4]:
from pyomo.environ import *
import numpy as np

# enter data as numpy arrays
A = np.array([[3, 4], [2, -3]])
b = np.array([26, -11])

# set of row indices
I = range(len(A))

# set of column indices
J = range(len(A.T))

# create a model instance
model = ConcreteModel()

# create x and y variables in the model
model.x = Var(J)

# add model constraints
model.constraints = ConstraintList()
for i in I:
    model.constraints.add(sum(A[i,j]*model.x[j] for j in J) == b[i])

# add a model objective
model.objective = Objective(expr = sum(model.x[j] for j in J))

# create a solver
solver = SolverFactory('glpk')

# solve
solver.solve(model)

# print solutions
for j in J:
    print("x[", j, "] =", model.x[j].value)

x[ 0 ] = 2.0
x[ 1 ] = 5.0
